# Read csvs

## Import 

In [1]:
import os
import csv
import pandas as pd
import glob
import numpy as np

In [2]:
## set directory path
directoryPath = ('/Users/alia/Desktop/csvs/*.csv')

In [3]:
files = glob.glob(directoryPath)

In [4]:
#create a df
frame = pd.DataFrame()
list_ = []

## Read through csv files

In [5]:
for read in files:
    reader = pd.read_csv(read,index_col=None, header=0, encoding = "ISO-8859-1")
    patient = read[-7:-4]

    #Extract what you need into dataframe
    #	Need: pt ID		ErrorON		ErrorOFF	stimON		stimOFF		stimdur		stim latency	Error Lat. (this # will be negative, take only rows with - values)
    reader['during_stim_pos'] = reader['stim_onset'].astype(float) - reader['s_onset'].astype(float)
    reader['during_stim_neg'] = reader['s_onset'].astype(float) - reader['stim_offset'].astype(float)
    reader['stim_effect'] = reader['s_offset'].astype(float) - reader['stim_onset'].astype(float)
    reader['ptID'] = patient
    frame = frame.append(reader)

frame["during_stim_for_pos"] = np.where(frame["during_stim_pos"] > 0, 'yes', 'no')
frame["during_stim_for_neg"] = np.where(frame["during_stim_neg"] < 0, 'yes', 'no')

## export all data into a csv 

In [10]:
#output column names in new csv file with extracted info
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","error2","region", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot"), axis=1)
# frame = frame.dropna()
#At the end of the for loop, write data to csv files (make sure to append)
frame.to_csv('all_patients.csv')

## export only trials with stim

In [ ]:
frame = frame[frame.stim == 'Y']
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","Unnamed: 14", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)

frame.to_csv('stim_trials_only.csv')

## export everything without nopot

In [ ]:
frame = frame[frame.timePlot != 'noplot']
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","Unnamed: 14", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)
frame.to_csv('without_noplot_stim_only.csv')

## export only motor

In [ ]:
frame = frame[frame.error == 'motor']
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","Unnamed: 14", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)
frame.to_csv('motor_only.csv')

## export only word 

In [ ]:
frame = frame[frame.error == 'word']
frame = frame.reindex_axis(("ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","Unnamed: 14", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)

frame.to_csv('word_only.csv')

## Anatomy and Error Type

In [13]:
##reindex frame 
frame1 = pd.DataFrame()
for i in frame:
    a = frame[i].tolist()
    frame1[i] = a

In [163]:
import pandas as pd

In [333]:
## find comorbid sites
patient = np.unique(frame1['ptID'])
# comorbid = pd.DataFrame()
comorbid_sites = []
co_index = []
for p in patient:
    motor = []
    word = []
    indx = np.where(frame1['ptID'] == p)
    for i in indx[0]:
        if frame1['error'][i] == 'motor':
            if frame1['stim_loc'][i] in motor: 
                pass
            else:
                motor.append(frame1['stim_loc'][i])
        if frame1['error'][i] == 'word':
            if frame1['stim_loc'][i] in word: 
                pass
            else:
                word.append(frame1['stim_loc'][i])
    for x in motor:
        if x in word:
            if str(x) == 'nan':
                pass
            else:
                comorbid_sites.append(str(x)+'_' + str(p))
print(comorbid_sites) 
print(co_index)
                


['A_119', 'A_147', 'A_152', 'A_155', 'B_159', 'B_170', 'A_232', 'A_A63', 'B_A71']
[]


In [330]:
sites = []
for c in co_index:
    sites.append(frame1['region'][c])

ventral_co = sites.count('ventral') 
dorsal_co = sites.count('ifg') 

print(sites)


['ventral', nan, nan, nan, 'ventral', nan, nan, 'dorsal', nan]


In [14]:
## counts how many errors are at a given site
def count(error,site):
    anatomy = pd.DataFrame()
    myidx = np.where(frame1['region'] == site)[0]
    errors = []
    for i in myidx:
        errors.append(frame1['error'][i])
        errors.append(frame1['error2'][i])
    counts = errors.count(error)
        
    return('number of ' + error + ' errors at site ' + site + ': ' + str(counts))

        

In [22]:
count('syllable','other')

'number of syllable errors at site other: 0'

## Calculating Stim Gap

In [90]:
framey = frame[frame.stim == 'Y']
framey = framey[framey.timePlot == 'noplot']
framey = framey[framey.error != '']




framey = framey.reindex_axis(("trial","ptID", "s_onset", "s_offset", "s_dur", "stim_onset", "stim_offset", "during_stim_pos", "during_stim_neg", "stim_effect", "error","error2", "during_stim_for_pos", "during_stim_for_neg", "stim_loc", "stim", "timePlot", "region"), axis=1)

frame1 = pd.DataFrame()
for i in framey:
    a = framey[i].tolist()
    frame1[i] = a

frame1['two_before_s_onset'] = np.zeros(len(frame1['s_onset']))
frame1['two_before_s_offset'] = np.zeros(len(frame1['s_onset']))
frame1['one_before_s_offset'] = np.zeros(len(frame1['s_onset']))
frame1['one_before_s_onset'] = np.zeros(len(frame1['s_onset']))
frame1['one_after_s_onset'] = np.zeros(len(frame1['s_onset']))
frame1['one_after_s_offset'] = np.zeros(len(frame1['s_onset']))


frame1['two_before_s_onset'][2:] = frame1['s_onset'][:-2]
frame1['two_before_s_offset'][2:] = frame1['s_offset'][:-2]

frame1['one_before_s_offset'][1:] = frame1['s_offset'][:-1]
frame1['one_before_s_onset'][1:] = frame1['s_onset'][:-1]

frame1['one_after_s_onset'][:-1] = frame1['s_onset'][1:]
frame1['one_before_s_onset'][:-1] = frame1['s_onset'][1:]




/Users/alia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/alia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set o

In [91]:
frame1['error']

0        motor
1        motor
2        motor
3         word
4         word
5        motor
6         word
7        motor
8        motor
9        motor
10       motor
11        word
12        word
13        word
14        word
15        word
16            
17        word
18        word
19       motor
20       motor
21        word
22       motor
23       motor
24       motor
25       motor
26       motor
27        word
28       motor
29       motor
        ...   
41        word
42    syllable
43        word
44       motor
45       motor
46        word
47       motor
48       motor
49        word
50        word
51       motor
52        word
53        word
54       motor
55       motor
56       motor
57        word
58        word
59        word
60       motor
61       motor
62       motor
63        word
64       motor
65        word
66       motor
67       motor
68       motor
69       motor
70       motor
Name: error, Length: 71, dtype: object

In [92]:
## find timing 

frame1['tbon'] = frame1['two_before_s_onset'][2:].astype(float) - frame1['stim_onset'].astype(float)
frame1['tboff'] = frame1['two_before_s_offset'][2:].astype(float) - frame1['stim_onset'].astype(float)

frame1['obon'] = frame1['one_before_s_onset'][1:].astype(float) - frame1['stim_onset'].astype(float)
frame1['oboff'] = frame1['one_before_s_offset'][1:].astype(float) - frame1['stim_onset'].astype(float)

frame1['son'] = frame1['s_onset'].astype(float) - frame1['stim_onset'].astype(float)
frame1['soff'] = frame1['s_offset'].astype(float) - frame1['stim_onset'].astype(float)

frame1['oaon'] = frame1['one_after_s_onset'][:-1].astype(float) - frame1['stim_onset'].astype(float)
frame1['oaoff'] = frame1['one_after_s_offset'][:-1].astype(float) - frame1['stim_offset'].astype(float)


frame1['before_gap'] = frame1['obon'] - frame1['tboff']

frame1['stim_gap'] = frame1['son'] - frame1['oboff']

frame1['after_gap'] = frame1['oaon'] - frame1['soff']









In [86]:
a = framey['trial'].tolist()
frame1['tiral'] = a
frame1 = frame1.reindex_axis(('tiral','ptID', 'before_gap', 'stim_gap', 'after_gap', "stim"), axis = 1)
frame1.to_csv('stim_info3.csv')


In [87]:
frame1

,tiral,ptID,before_gap,stim_gap,after_gap,stim
0,21,103,NaN,NaN,12.279,NaN
1,23,103,NaN,12.279,6.015,NaN
2,24,103,91.661,6.015,70.949,NaN
3,37,103,135.151,70.949,53.460,NaN
4,50,103,161.302,53.460,32.940,NaN
5,72,103,64.635,32.940,-23.668,NaN
6,52,119,-170.934,-23.668,-182.179,NaN
7,6,133,-120.008,-182.179,84.810,NaN
8,23,133,-13.367,84.810,82.223,NaN
9,31,133,193.650,82.223,22.750,NaN
